<a href="https://colab.research.google.com/github/eodnsl41/abcd/blob/main/Neural_Net_%26_back_propagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Affine layer : Hidden layer끼리 연결

import numpy as np
class Affine:
    def __init__(self, W, b):
        self.W = W #변수 설정. weight, bias
        self.b = b

        self.x = None
        self.original_x_shape = None
        #가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b #x가 들어오면 dot product 연산을 함

        return out

    def backward(self, dout): #역전파. 
        dx  = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)

        dx = dx.reshape(*self.original_x_shape)
        return dx


In [3]:
#softmax는 값을 정규화 시킨다.
#전체의 합을 1로 만드는게 softmax의 주된 목적

def softmax(x):
    if x.ndim == 2: #ndim : 앞에 있는 matrix의 차원확인
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis = 0)
        return y.T
    x = x - np.max(x) #너무 큰 값이 나오지 않기 위해 max값을 뺌

    return np.exp(x) / np.sum(np.exp(x))


In [4]:
#많은 activation functions 중 ReLU 함수 정의하는 방법

# 순전파때는 0 이상일때만 인풋 그대로 아웃풋을 해주는게 렐루함수
#0 이하일 때는 그냥 값을 0으로 해주는게 순전파 때 역할
# 역전파때는 그냥 그대로 넣어주면 됨

class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx


In [5]:
from numpy.matrixlib.defmatrix import N
#one-hot vector

def cross_entropy_error(y,t): 
    #y값 : 신경망 예측한 y 프레딕션 값
    #t값 : 우리가 갖고 있는 라벨링 데이터 값
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        #에러 방지 조건

    #훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t= t.argmax(axis =1)

    batch_size = y.shape[0]
    return - np.sum(np.log(y[np.arrange(batch_size), t] + 1e-7)) / batch_size
    #10의 -7승 더해준 이유 : log0이 존재하면 에러가 걸리니까 작은값이라도 더해주는거



class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None #손실함수
        self.y = None #softmax의 출력
        self.t = None #정답레이블 (원-핫 인코딩의 형태)

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        #비교를 해야하니까 라벨데이터 갖고옴

        return self.loss

    def backward(self, dout=1): 
        #역전파때에는 이거에 미분을 한 거를 비교를 해서 전달을 하게 됨
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size:
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arrange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx


In [6]:
#실제 미분값 구해서 학습 시키는것 / 역전파때 함수 구하는 것 -> 비교해봄

def numerical_gradient(f, x):
    h= 1e-6 #0.000001 #굉장히 작은 값
    grad = np.zeros_like(x)

    it = np.nditer(x, flags = ['multi_index'], op_flags =['readwrite']) #np.nditer는 편미분. 인풋값에 따라서 편미분
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) #f(x+h)

        x[idx] = tmp_val - h
        fxh2 = f(x) #f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)

        x[idx] = tmp_val #값 복원
        it.iternext()

    return grad

In [7]:
#feedforward network 정의

from collections import OrderedDict #딕셔너리는 순서가 없는데 순서를 만든거임
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        #가중치 초기화
        self.params = {}
        self.params['W1']= weight_init_std * np.random.randn(input_size, hidden_size) #weight의 초기값을 랜덤하게 정의한 것
        self.params['b1']= np.zeros(hidden_size) 
        self.params['W2']= weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2']= np.zeros(output_size)

        #계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1']) #affine은 히든레이어
        self.layers['Relu1'] = Relu() #relu function 하나 넣음
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x): #신경망 예측치를 만드는게 prediction 함수
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    # x : 입력데이터, t:정답레이블

    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y,t) #순전파 사용

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis = 1)
        if t.ndim != 1 : t = np.argmax(t, axis =1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    def numerical_gradient(self, x, t):
        loss_W = lambda W : self.loss(x,t)

        grads ={}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        #forward
        self.loss(x,t)

        #backward
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        #결과 저장
        grads ={}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

In [ ]:
#mnist
#~~